# Chapter 24 - NeuralProphet

In [ ]:
#!pip install gluonts[torch]

## Listing 24-1. Importing the data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

y = pd.read_csv('heatwave-pageviews-20150701-20250531.csv')
y.columns = ['date', 'y']
y['date'] = pd.to_datetime(y['date'])
y = y[:-5]

## Listing 24-2. Convert the data to ListDatasets

In [ ]:
from gluonts.dataset.common import ListDataset
import pandas as pd

start = pd.Timestamp("01-07-2015", unit="h")
train_ds = ListDataset([{'target': y[:-12]['y'].values, 'start': start}], freq='ME')
test_ds = ListDataset([{'target': y[-12:]['y'].values, 'start': start}],freq='ME')


## Listing 24-3. Fitting the default DeepAR model

In [ ]:
from gluonts.torch import DeepAREstimator
import numpy as np
np.random.seed(7)

estimator = DeepAREstimator(
    prediction_length=12,
    context_length=12,
    freq='ME'
)

predictor = estimator.train(train_ds)

## Listing 24-4. Estimating the forecast

In [ ]:
predictions_generator = predictor.predict(test_ds)

forecast = next(predictions_generator)

# Get the 50th percentile of the forecast (median)
predictions = forecast.quantile(0.5)

## Listing 24-5. Evaluating the prediction on the test set

In [ ]:
test_values = test_ds[0]['target']
fcst_values = forecast.samples.mean(axis=0)

import matplotlib.pyplot as plt
plt.plot(test_values)
plt.plot(fcst_values)
plt.legend(['test', 'forecast'])
plt.show()

## Listing 24-5. Evaluating the prediction on the test set

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mape_metric = 1-mean_absolute_percentage_error(test_values, fcst_values)
print(mape_metric)

## Listing 24-6. Hyperparameter tuning using Ray

In [ ]:
#!pip install ray[tune]
#!pip install hyperopt

from gluonts.torch import DeepAREstimator
import matplotlib.pyplot as plt

from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch

# Define the tune function
def tune_deepar(config):
    estimator = DeepAREstimator(
        prediction_length=12,
        context_length=config["context_length"],
        freq='ME',
        hidden_size=config["hidden_size"],
        num_layers=config["num_layers"],
        dropout_rate=config["dropout_rate"],
        batch_size=config["batch_size"]
    )

    predictor = estimator.train(train_ds)
    predictions_generator = predictor.predict(test_ds)
    forecast = next(predictions_generator)
    fcst_values = forecast.samples.mean(axis=0)
    mape_metric = 1 - mean_absolute_percentage_error(test_values, fcst_values)

    tune.report({'mape':mape_metric})

# Define the search space
search_space = {
    "context_length": tune.choice([3, 6, 12]),
    "hidden_size": tune.choice([20, 30, 40]),
    "num_layers": tune.choice([2, 3]),
    "dropout_rate": tune.uniform(0.0, 0.1),
    "batch_size": tune.choice([16, 64, 128])
}

# Set up the HyperOptSearch
search_alg = HyperOptSearch(metric="mape", mode="min")

# Run the tuning process
analysis = tune.run(
    tune_deepar,
    config=search_space,
    num_samples=10,  # Number of trials to run
    search_alg=search_alg,
    verbose=1,
    resources_per_trial={"cpu": 2, "gpu": 0}
)

## Listing 24-7. Plot the best model

In [ ]:
# You can now use the best configuration to train the final model

estimator_tuned = DeepAREstimator(
    prediction_length=12,
    context_length=3,
    freq='ME',
    hidden_size=20,
    num_layers=2,
    dropout_rate=0.0745556,
    batch_size=64
)

predictor_tuned = estimator_tuned.train(train_ds)
predictions_generator_tuned = predictor_tuned.predict(test_ds)
forecast_tuned = next(predictions_generator_tuned)
fcst_values_tuned = forecast_tuned.samples.mean(axis=0)


mape_metric_tuned = 1 - mean_absolute_percentage_error(test_values, fcst_values_tuned)
print("MAPE (tuned):", mape_metric_tuned)


plt.plot(test_values)
plt.plot(fcst_values_tuned)
plt.legend(['test', 'forecast (tuned)'])
plt.show()